In [1]:
import sys
import os

fin_app_path = os.path.abspath('..')
sys.path.append(fin_app_path)

In [2]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader

from app.consts import l_llm
from app.chains_rag import get_excerpt_chain
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate 
# from langchain_core.output_parsers import StrOutputParser


# from langchain_core.pydantic_v1 import BaseModel, Field
# from app.consts import default_prompt


load_dotenv()

True

In [10]:
LLM_s = l_llm['gpt-4o-mini']
LLM_l = l_llm['gpt-4o']

In [3]:

# path = 'rag_docs/BHARTIARTL_AR_22.pdf'
path = r'C:\Users\vivek.phuloria\Documents\side_projects\fin5\rag_docs\BHARTIARTL_AR_22.pdf'

loader = PyPDFLoader(path)
pages = loader.load()



In [4]:
s = pages[24].page_content
print(s)

23
Bharti Airtel Limited
was the first to launch 4G in India, and under his leadership, 
Airtel expanded its 4G services across the Country.
  Mr. Gopal Vittal has been elected to the Board of GSMA 
for the 2019-20 term. He has joined 25 other distinguished 
leaders from across the world to provide strategic guidance 
and roadmap to the premier global telecoms body, which 
represents more than 750 of the world’s mobile operators.
(d)  Comparative remuneration profile with respect 
to industry, size of the Company, profile of the 
position and person (in case of expatriates the 
relevant details would be with respect to the 
country of origin):
  Taking into consideration the size of the Company, his 
profile, the responsibilities shouldered by him and the 
industry benchmarks, the remuneration proposed to be 
paid is commensurate with the remuneration packages 
paid to similar senior levels in other companies.
(e)  Pecuniary relationship directly or indirectly with 
the company, or rel

In [ ]:
from langchain.prompts import ChatPromptTemplate 
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from app.consts import l_llm
from typing import Literal, List
from pydantic import BaseModel, Field
from app.consts import create_prompt

def f_get_excerpt_chain():
    class Excerpt(BaseModel):
        """ 
        An excerpt with it's tag
        """ 
        excerpt  : str = Field(description = "A small passage from a company's document")
        tag      : Literal["Operational Metrics",  "Financial Metrics",  "Key Personell",  "Product Strategy",  "Expansion" , "Competition",  "Sectoral Phenomenon",  "Regulatory", "Macro Factors",  "Organizational Details",  "MnA", "Alliances"] = Field(description = "Tag of the passage")

    class l_Excerpt(BaseModel):
        """List of excerpt and  tags""" 

        l_excerpt : List[Excerpt] = Field("List of excerpt and  tags")
         

    excerpt_prompt = r"""
    You are a business analyst extracting relevant information from a passage of a company's annual report.
    Note that given the parsing process of the document, some unneccessary newline or whitespaces may have been added in the passage.
    You have been provided with a list of important tags below along with their description. 
    Go through the contents of the input provided to you, and  find the exact excerpt which talks about these tags.
    Your goal is to to extract all relevant excerpts , but for a single excerpt, the whole thing should be talking about just a single tag. 
    Each excerpt can range in length from a phrase to a sentence or at max 2-3 sentences, and should be as short as possible while making semantic sense.
    For each excerpt identify the most relevant tag. Only use the tags from the below list.
    Your response should the different excerpts with their related tag, formatted as a list of tuples
    Sample response:
    \[
        \{ "excerpt": "The Company has purchased additional 4.7% equity interest in org X from org Y", "tag": "MnA"},
        \{ "excerpt": "In partnership with org X, the company has conducted India's first 5G trial in the 700 MHz band.", "tag": "Alliances"},
        \{ "excerpt": "Company's X division launched feature Y, a digital platform for customers to be able to do Z", "tag": "Product Strategy"},
        \{ "excerpt": "The Company had 208.4 million data customers at the end of March 31, 2022, of which 200.8 million were segment X customers.", "tag": "Operational Metrics"},
        \{ "excerpt": "Consequently, EBIT for the year was 248,531 Mn, increasing by 49.6% and resulting in a margin of 21.3% vis-a-vis 16.5% in the previous year.", "tag": "Financial Metrics"}
    ]
    If the page talks about none of these tags, respond with an empty list [].

    List of Tags to choose from
    - Operational Metrics: Any excerpt that talks about the companies operational metrics for the business, eg. Average Revenue for User, Churn Rate, Net Promoter Score, Resoliton Time etc. Basically anything that indicates how well the company is serving its customers.
    - Financial Metrics: Any excerpt talking about the companies financia; metrics for the business, eg.any Revenue, Costs, Profits, Debts, Valuation, Share Price etc.
    - Key Personell: Any excerpt talking about any executive, management or key personell like a CxO,MD, board of directors etc. - any announcement regarding a change in the personell, modification in their role. or their achievement. 
    - Product Strategy: If the excerpt talks about it's companies portfolio of products or offerings. If any particular product is doing very well or poorly, or if they have changed the product portfolio - adding or removing some items, or changing their focus areas.
    - Expansion: If the excerpt is talking about expansion plans of the organization. This could be by capital expenditure, vertical growth by entering into some other part of the value chain, horizontal growth by targetting a different product line or customer segment, or entering a new business line. This could be an organic growth with internal team working on the new initiative, or inorganic growth via by Mergers or Acquisitions.
    - Competition: If the excerpt talks about it's competitor or the compeitive landscape for its industry.
    - Sectoral Phenomenon: If the excerpt talks about any wholesale changes in consumer preferences; price changes in key ingredients like commodities, energy, labour ; technological advancements 
    - Regulatory: If the excerpt talks about the regulatory landscape, or if the government or any other governing body's policy has impacted the company's business in any way.
    - Macro Factors: If the excerpt talks about any macro phenomena that impacts the business - like global politics, trade treaties, war etc. Also if the excerpt talks about country's economics and populations well being 
    - Organizational Details: If the excerpt talks about the organization structure, or about any parent or subsidiary organization.
    - MnA: If the excerpt talks about any mergers or acquisisions that the company has undertaken.
    - Alliances: If the company has entered into a strategic alliance or partnership with any other organization
    """.strip()
    llm_l_st = LLM_l.with_structured_output(l_Excerpt)
    # chain = create_prompt(excerpt_prompt) | llm_l_st
    chain = create_prompt(excerpt_prompt)

    return chain, excerpt_prompt





In [28]:
excerpt_chain, prompt = f_get_excerpt_chain()
# res = excerpt_chain.invoke(s)

create_prompt = lambda system_prompt : ChatPromptTemplate.from_messages([('system',system_prompt),('human','{input}')])
create_prompt(prompt)

ChatPromptTemplate(input_variables=['"excerpt"', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['"excerpt"'], input_types={}, partial_variables={}, template='You are a business analyst extracting relevant information from a passage of a company\'s annual report.\n    Note that given the parsing process of the document, some unneccessary newline or whitespaces may have been added in the passage.\n    You have been provided with a list of important tags below along with their description. \n    Go through the contents of the input provided to you, and  find the exact excerpt which talks about these tags.\n    Your goal is to to extract all relevant excerpts , but for a single excerpt, the whole thing should be talking about just a single tag. \n    Each excerpt can range in length from a phrase to a sentence or at max 2-3 sentences, and should be as short as possible while making semantic sense.\n    For each e